In [112]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [8]:
tokenizer= AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model= AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')



C:\Users\ulas0\anaconda3\envs\diffusion\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ulas0\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [34]:
text = "Only english."
encoded_input = tokenizer.encode(text, return_tensors='pt')
encoded_input[0]
#tokenizer.decode(encoded_input[0][11])

tensor([  101, 10104, 10819, 10208, 94663, 10112,   119,   102])

In [35]:
result=model(encoded_input)

In [36]:
result # the position with the highest score represents the rating given. For examplei this is a comment with rating one.

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2015, -0.6552, -0.2172,  0.0088,  0.7561]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [37]:
score= int(torch.argmax(result.logits)) + 1

In [182]:
### Collect reviews from a site

r = requests.get('https://www.trustpilot.com/review/www.toluna.com') #scrap any company you want from trustpilot
soup= BeautifulSoup(r.text, 'html.parser')
regex= re.compile('.*typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn.*') #extract the comment tags from the website
results=soup.find_all('p', {'class':regex})
reviews=[k.text for k in results]

In [183]:
reviews

["Toluna is consistent in providing me relevant surveys. I have been enjoying your site even i don't get to complete a survey by playing games - wheel and match games, and voting in the quick surveys. Just two things for Toluna to look into: faster redemption process (3 days would be nice) and coming up with contests. By the way, i noticed lately that there is a survey compensation of 50 ( to 70 points) for uncompleted surveys. Thank you for this added feature as well. Keep up the good work. ",
 'Just started using it. I like that you still get points when you don’t qualify for a survey, however there seems to be a large amount of surveys I do not qualify for. In addition, the “recommended” surveys aren’t always still available. Again, I just started using and 4 surveys I tried to complete were no longer available. I have only qualified and completed 1 full survey out of the many I’ve tried.',
 'You got four because of the numbers of e-mails I get claiming to tell me about an available

In [184]:
df= pd.DataFrame(np.array(reviews), columns=['review'])
df=df.iloc[1:]
df= df.iloc[::2, :]

In [185]:
def sentiment_score(review):
    encoded_input = tokenizer.encode(review, return_tensors='pt')
    result=model(encoded_input)
    score= int(torch.argmax(result.logits)) + 1
    return score


In [189]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:1024])) #maximum 512 tokens per comment for BERT

In [190]:
df['review']

1     Just started using it. I like that you still g...
3     Most of the survey invitations are genuine (un...
5     The platform  has fun and relevant surveys tha...
7     Toluna suspended my account without me being a...
9     Toluna is the best survey site around, they ha...
11    it's great even i am always failed to proceed ...
13    Toluna have brilliant and interesting surveys....
15    TOLUNA IS ,I FIND OTHER SURVEY KINDS BUSINESS....
17    Would be good if on completion of surveys the ...
19    Love being part of this able to have a voice, ...
Name: review, dtype: object

In [191]:
df

,review,sentiment
1,Just started using it. I like that you still g...,3
3,Most of the survey invitations are genuine (un...,4
5,The platform has fun and relevant surveys tha...,5
7,Toluna suspended my account without me being a...,1
9,"Toluna is the best survey site around, they ha...",4
11,it's great even i am always failed to proceed ...,5
13,Toluna have brilliant and interesting surveys....,5
15,"TOLUNA IS ,I FIND OTHER SURVEY KINDS BUSINESS....",2
17,Would be good if on completion of surveys the ...,3
19,"Love being part of this able to have a voice, ...",5
